# Opentrons Protocol for AFB Epistasis Mutant Cloning

First we will import python API (Application Programming Interface) into our script so that we can develop a protol that can be understood by the robot


In [6]:
!pip install --upgrade opentrons

Requirement already up-to-date: opentrons in c:\users\jonbr\anaconda3\lib\site-packages (3.21.2)


Metadata is a dictionary of data that is read by the server and returned to the opentrons app. give yourself credit.
you are however required to specify the 'apiLevel' here

In [5]:
from opentrons import protocol_api
metadata = {
    'protocolName': 'AFB Epistasis Mutations Protocol',
    'author': 'John Bryant <jbryant2@vt.edu>',
    'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants',
    'apiLevel': '2.6'
}
print(metadata)

{'protocolName': 'AFB Epistasis Mutations Protocol', 'author': 'John Bryant <jbryant2@vt.edu>', 'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants', 'apiLevel': '2.6'}


Protocols are structured around a function called run(protocol). The function has to be name "run" and must take exactly one mandatory argument ("protocol"). function run is a container for the code that defines your protocol. 

protocol context has 2 responsibilities:
1)remember, track, and check robot's state
2)expose the functions that make the robot execute actions.

# Before Running this code, define the following variables: 

In [40]:
#variables:
#primer dilutions:
stkprm = 100 #concentration of the stock primer you are adding
stkvol = 1 #the volume of stock primer you are adding
dilprm = 2.5 #this is the concentration in uM that you want your working dilution to be

#template dilutions:
stktemp = [157,239,301,198] #input the concentration of each template in this vector in ng/uL
stkv = 1 #the volume of template you are adding
diltemp = 12.5 #the concentration in ng/uL that you want your working template to be

#pcr reaction
Numprimers = 4 #this is how many primers go in each pcr reaction.
primerconcentration = 0.1 #this is the concentration you want each primer to be in the pcr reaction
pcrvol = 25 #this is the total volume of your pcr reaction 
templatengs = .5 #this is the concentration of template you want in your pcr rxn in ng/uL

In [41]:
#dilvol = ? #this is the volume you bring you stock primer up to in order to make the working dilution

dilvol = (stkprm*stkvol)/dilprm #this is the calculation for volume you bring you stock primer up to based on C1V1=C2V2
H2OAdd = dilvol-stkvol #this is the amount of H2O that you add to your stock primer. since all stock primers are 100uM
                       #concentrations when you start, there is no need to vectorize the H2OAdd variable because it is the
                       #same for each primer.
print (H2OAdd)

39.0


In [42]:
import numpy as np
dilv = np.zeros(len(stktemp)) #initiate a the list of volumes that each stk template in the stktemp vector need to be brought up to. A for loop
          #will be used to solve for each of these.

for i in range(len(stktemp)):
    dilv[i] = (stktemp[i]*stkv)/diltemp #this solves for the volume each template in stktemp needs to be brought up to
print('bring each template up to',dilv, '\n')

H2OAddtemp = np.zeros(len(stktemp))
for i in range(len(stktemp)):
    H2OAddtemp[i] = dilv[i] - stkv
print('to do this add', H2OAddtemp, 'water to each template','\n')


bring each template up to [12.56 19.12 24.08 15.84] 

to do this add [11.56 18.12 23.08 14.84] water to each template 



In [43]:
#PCR Reaction

Amt2addfor1prm = (primerconcentration*pcrvol)/dilprm #this is the amount of primer that you need to add to get the right primer concentration
Amt2addforallprm = Amt2addfor1prm * Numprimers #this is the total amount of primer you will be adding based on number of primers in your rxn
print('you will add', Amt2addfor1prm, 'uL of each primer and', Amt2addforallprm, 'total')
      
Amt2addtemp = (templatengs*pcrvol)/diltemp #this is the amount of template you will add
print('you will add', Amt2addtemp, 'uL of template')

Q5 = pcrvol/2
pcrtotalvolume = Q5 + Amt2addforallprm + Amt2addtemp
print('you will add', Q5, 'uL of Q5')

wawa = pcrvol - pcrtotalvolume
print('add', wawa, 'uL water')

if Amt2addfor1prm < 1:
    print('Dilute primers more in initial dilution')
if Amt2addforallprm > 5:
    print('make primer concentration higher in initial dilution')

you will add 1.0 uL of each primer and 4.0 total
you will add 1.0 uL of template
you will add 12.5 uL of Q5
add 7.5 uL water


In [12]:
def run(protocol: protocol_api.ProtocolContext):
    
    #labware:
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', '1')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_20ul','2')
    tuberack1 = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap','3') #holds stock primers and templates
    bigtuberack = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical','4') #holds molec bio grad H2O
    tuberack2 = protocol.load_labware('opentrons_24_tuberack_nest_1.5ml_snapcap','5') # holds dilute primers and templates
    
    tc_mod = protocol.load_module('Thermocycler Module')
    plate = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    
    #can put dpnI digestion on either place 6 or 9
    
    
    #pipettes
    right_pipette = protocol.load_instrument('p300_single','right',tipracks=[tiprack1])
    left_pipette = protocol.load_instrument('p20_single','left',tipracks=[tiprack2])
    
    #commands
    
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(H2OAdd, bigtuberack.wells('C5'), tuberack2.wells('A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C1','C2','C3','C4','C5','C6','D1','D2','D3'))  # dilutent
 
    # loop through each tuberack well to dilute stock primers
    for i in range(len(stktemp)):

      # save the source well and destination column to variables
      source = tuberack1.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
    
    #need to loop through and dilute both of the templates (using variable B defined above)
    p200.distribute(h20totemp, bigtuberack.wells('C5'), tuberack2.wells('D4','D5'))  # dilutent
 
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(B,tuberack1['D5'])
    left_pipette.dispense(B,tuberack2['D5'])
    left_pipette.drop_tip()
    #########################################################################################
    
    #time to start filling up the pcr tubes
    
    #first add appropriate amount of water
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(amth20, bigtuberack.wells('C5'), plate.wells('A1','A2','A3','A4','A5','A6','A7','A8','A9'] # dilutent
 
    
    #move diluted primers into pcr tubes how do i do this with sets of four primers going into each pcr tube
    # loop through each tuberack well to dilute diluted primers
    for i in range(21):

      # save the source well and destination column to variables
      source = tuberack2.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
      #move diluted templates into pcr tubes - this should probable be done with an if else statement
                                                                 
    
   
    #######################################################################################################
                                                                 
    #set up pcr reaction in the thermocycler module
    
    #open lid
    tc_mod.open_lid()
    
    #close lid
    tc_mod.close_lid()
    
    #set lid temperature
    tc_mod.set_lid_temperature(100)
    
    #set block temp profile
   
    tc_mod.execute_profile(98, hold_time_seconds=30, block_max_volume=50)

    profile = [
    {'temperature': 98, 'hold_time_seconds': 10},
    {'temperature': 65, 'hold_time_seconds': 30},
    {'temperature': 72, }]

    tc_mod.execute_profile(steps=profile, repetitions=34, block_max_volume=50)
    
    tc_mod.execute_profile(72, hold_time_minutes=5, block_max_volume=50)
    
    ################################################################################
    #add DpnI for digestion
    #add DpnI to each PCR product
    
    

    

first we need dilute the stock primers

start off by adding water

In [ ]:
#commands
    
#put water in for diluted primers by using nested for loop and H2OAdd vector 
Rows = ('A','B','C','D')
columns = ('1','2','3','4','5','6')

for i in Rows:
    for j in columns:
        
        if i+j = 'D4'
            break
        
        # save the source well and destination column to variables
        source = bigtuberack('C5')
        destination = tuberack2.wells(i+j)
        
       
            
        left_pipette.pick_up_tip()
        left_pipette.aspirate(H2OAdd,source)
        left_pipette.dispense(H2OAdd,destination)
        left_pipette.drop_tip()
        

Now add in the stock primer

In [ ]:
for i in Rows:
    for j in columns:
        
        source = tuberack1.wells(i+j)
        destination = tuberack2.wells(i+j)
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(stkvol,source)
        left_pipette.dispense(stkvol,destination)
        left_pipette.drop_tip()
        

Now dilute the template. 

In [ ]:
source = tuberack1.wells

#put water in for templates by using nested for loop and H2OAdd vector 
Rowstemp = ('D')
columnstemp = ('4','5')

for i in Rowstemp:
    for j in columnstemp:
        
        if i+j = 'D6'
            break
        
        # save the source well and destination column to variables
        source = tuberack1.wells(i+j)
        destination = tuberack2.wells(i+j)
        
 #specifying how much water to add to each well, might be good to use a dictionary here

        if i+j == 'D4':
            w = 1
        if i+j == 'D5':
            w = 2
            
        left_pipette.pick_up_tip()
        left_pipette.aspirate(H2OAddtemp[w],source)
        left_pipette.dispense(H2OAddtemp[w],destination)
        left_pipette.drop_tip()


In [ ]:
for i in Rowstemp:
    for j in columnstemp:
        
        source = tuberack1.wells(i+j)
        destination = tuberack2.wells(i+j)
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(stkv,source)
        left_pipette.dispense(stkv,destination)
        left_pipette.drop_tip()
        

Now we have to set up the PCR reactions with the diluted primers and templates set up in previous steps

In [ ]:
#for AFB epistasis we will have 9 pcr reactions

Rowspcr = ('A')
columnspcr = ('1','2','3','4','5','6','7','8','9')

for i in Rowspcr:
    for j in columnspcr:
        
        source = tuberack2.wells




In [ ]:
assemblyrxns = [0,1,2,3,4,5,6,7,8]
assemblytubes = ['A1','A2','A3','A4','A5','A6','A7','A8','A9']

#put water in each one
for i in assemblytubes:
    source = bigtuberack.wells('C5')
    destination = plate.wells(i)
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(H2OAdd,source)
    left_pipette.dispense(H2OAdd,destination)
    left_pipette.drop_tip()

#put template in each one


#put primer in each one
for i in assemblyrxns:
    
    listofprimerlocations = Assembly2wells[str(i)][0]
    
    for j in len(Numprimers):
        
        source = tuberack2.wells(listofprimerlocations[j])
        destination = plate.wells(assemblyrxn2pcrtube[str(i)])
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(Amt2addfor1prm,source)
        left_pipette.aspirate(Amt2addfor1prm,destination)
        left_pipette.drop_tip()

#put Q5 in each one
for i in assemblytubes:
    
    source = ?????????.wells('??')
    destination = plate.wells(i)
    